In [2]:
# !tar -xf aclImdb_v1.tar.gz

In [3]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

In [13]:
# import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Convert training data into csv file

In [4]:
import os
import pandas as pd

pathNeg = "aclImdb/train/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "aclImdb/train/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('train.csv')

Convert testing data into csv file

In [5]:
pathNeg = "aclImdb/test/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "aclImdb/test/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('test.csv')

convert vocab into csv file

In [7]:
with open("aclImdb/imdb.vocab", encoding='utf8') as f:
  vocab = []
  for line in f:
    vocab.append(line.rstrip())

vocab_df = pd.DataFrame(vocab, columns=['vocab'])
print(vocab_df)
vocab_df.to_csv('vocab_given.csv', index=False)

                   vocab
0                    the
1                    and
2                      a
3                     of
4                     to
...                  ...
89522          copywrite
89523             artbox
89524          kinky-sex
89525           urrrghhh
89526  investigator-like

[89527 rows x 1 columns]


In [8]:
from nltk.tokenize import word_tokenize

def tokenizeReview(review):
  review = review.replace('.', ' . ')
  tokens = word_tokenize(review.lower())
  return tokens

Get vocab from training set

In [14]:
from collections import Counter
reviews = train_df['text'].tolist()
reviews = list(map(tokenizeReview, reviews))
reviews = [item for review in reviews for item in review]
vdict = Counter(reviews).most_common()

In [15]:
vocabs = []
for pair in vdict:
  vocabs.append(pair[0])

vocab_df = pd.DataFrame(vocabs, columns=['vocab'])
vocab_df.to_csv('vocab.csv', index=False)
print(vocab_df)

               vocab
0                the
1                  .
2                  ,
3                and
4                  a
...              ...
100550    people-and
100551         padre
100552   unmatchable
100553  scenery-wise
100554  non-teenager

[100555 rows x 1 columns]


Read training data and testing data from csv file

In [16]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
vocab_df = pd.read_csv('vocab.csv')
print(vocab_df)

               vocab
0                the
1                  .
2                  ,
3                and
4                  a
...              ...
100550    people-and
100551         padre
100552   unmatchable
100553  scenery-wise
100554  non-teenager

[100555 rows x 1 columns]


In [17]:
vocab_dict = pd.Series(vocab_df.index + 1, index=vocab_df.vocab).to_dict()
print(vocab_dict)

{'the': 1, '.': 2, ',': 3, 'and': 4, 'a': 5, 'of': 6, 'to': 7, 'is': 8, '/': 9, '>': 10, '<': 11, 'br': 12, 'it': 13, 'in': 14, 'i': 15, 'this': 16, 'that': 17, "'s": 18, 'was': 19, 'as': 20, 'for': 21, 'with': 22, 'movie': 23, 'but': 24, 'film': 25, ')': 26, '(': 27, '``': 28, 'you': 29, 'on': 30, "n't": 31, 'not': 32, "''": 33, 'are': 34, 'he': 35, 'his': 36, 'have': 37, 'be': 38, 'one': 39, '!': 40, 'all': 41, 'at': 42, 'they': 43, 'by': 44, 'an': 45, 'who': 46, 'from': 47, 'so': 48, 'like': 49, 'there': 50, 'her': 51, 'or': 52, 'just': 53, 'do': 54, 'about': 55, 'has': 56, 'if': 57, 'out': 58, '?': 59, 'what': 60, 'some': 61, 'good': 62, 'more': 63, 'when': 64, 'she': 65, 'very': 66, 'would': 67, 'even': 68, 'my': 69, 'no': 70, 'up': 71, 'time': 72, 'can': 73, 'which': 74, 'only': 75, 'story': 76, 'really': 77, 'had': 78, 'their': 79, 'see': 80, 'were': 81, 'we': 82, 'me': 83, 'did': 84, 'does': 85, ':': 86, "'": 87, 'than': 88, '-': 89, 'well': 90, 'much': 91, 'could': 92, 'been':

In [18]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vocab_dict:
      vec.append(int(vocab_dict[word]))
    else:
      vec.append(0)
  return vec

In [19]:
train_df['tokens'] = train_df['text'].apply(tokenizeReview)

In [20]:
train_df['vec'] = train_df['tokens'].apply(convertToIndex)

In [21]:
train_df

,id,text,score,tokens,vec
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 141, 46, 56, 7522, 1382, 21, 5, 451..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[646, 5007, 306, 1, 106, 3768, 1072, 9205, 6, ..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 225, 1, 9589, 902, 310, 28, 102, 128, 2, ..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 337, 3188, 7722, 18, 34406, 8..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16026, 212, 511, 56, 3152, 1785, 396, 5, ..."
...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, :, some, inventive...","[60, 18, 62, 55, 13, 86, 61, 4389, 4, 2034, 93..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 167, 82, 154, 93, 162, 187, 212, 25, 17, ..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 125, 16, 23, 6, 13592, 4..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 132, 23, 137, 13, 56, 5, 185, 6..."


In [22]:
print('Maximum review length: {}'.format(
len(max((train_df['tokens']), key=len))))
print('Minimum review length: {}'.format(
len(min((train_df['tokens']), key=len))))

Maximum review length: 2820
Minimum review length: 11


In [23]:
max_words = 500
dataset = sequence.pad_sequences(train_df['vec'], maxlen=max_words)
y = train_df['score']

In [24]:
X_train, X_val, y_train, y_val = train_test_split(dataset, y, test_size=0.3, stratify=y, random_state = 42)

In [25]:
vocab_size = len(vocab_dict) + 1
print(vocab_size)

100555


In [30]:
from keras import Sequential
from keras.layers import Embedding, Dense, Dropout, SimpleRNN
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(SimpleRNN(50, input_shape=(X_train.shape[1],1), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           3217760   
                                                                 
 simple_rnn (SimpleRNN)      (None, 50)                4150      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,221,961
Trainable params: 3,221,961
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 84s 150ms/step - loss: 104.6929 - accuracy: 0.5668
Epoch 2/5
547/547 [==============================] - 82s 149ms/step - loss: 0.6007 - accuracy: 0.6625
Epoch 3/5
547/547 [==============================] - 81s 149ms/step - loss: 0.4814 - accuracy: 0.7709
Epoch 4/5
547/547 [==============================] - 80s 147ms/step - loss: 0.3133 - accuracy: 0.8873
Epoch 5/5
547/547 [==============================] - 90s 164ms/step - loss: 0.1904 - accuracy: 0.9378


In [32]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8258666396141052


In [33]:
model.save('basic_simple_rnn_model')

INFO:tensorflow:Assets written to: basic_simple_rnn_model/assets


INFO:tensorflow:Assets written to: basic_simple_rnn_model/assets
